In [85]:
import typing
from dataclasses import dataclass
from typing import get_type_hints
from inspect import getfullargspec

from icecream import ic


@dataclass(frozen=True)
class Param:
    name: str
    annotation: typing.Type[type]
    type: str


def get_arg_types(function) -> typing.Optional[list[Param]]:
    try:
        fullspec = getfullargspec(function)
        typehints = get_type_hints(function)
    except TypeError:
        return None

    getparamtype = lambda param: typehints.get(param, None) or fullspec.annotations.get(param, None)

    types = [Param(param, getparamtype(param), 'positional') for param in fullspec.args]
    if fullspec.varargs is not None:
        types.append(Param('*' + fullspec.varargs, getparamtype(fullspec.varargs), 'varargs'))

    return types

# """FullArgSpec(
#     args=['a', 'b'],
#     varargs='args',
#     varkw='kwargs',
#     defaults=None,
#     kwonlyargs=[],
#     kwonlydefaults=None,
#     annotations={'a': 'int', 'b': 'int', 'args': 'None', 'kwargs': 'str'}
# )"""

In [176]:
class CastError:
    def __init__(self, e):
        self.e = e

    def __repr__(self):
        return f'<ERROR: {self.e}>'


def process_arg_types(function, args: str):
    types = get_arg_types(function)
    out_args = []
    i = 0

    def trycastarg(arg, param):
        try:
            return param.annotation(arg)
        except Exception as e:
            return CastError(e)

    for arg, param in zip(args, types):
        if param.type == 'positional':
            if param.annotation is not None:
                out_args.append(trycastarg(arg, param))
            else:
                out_args.append(arg)
            i += 1

    vararg_type = next((p for p in types if p.type == 'varargs'), None)
    if vararg_type is not None:
        if vararg_type.annotation is not None:
            out_args.extend(map(lambda x: trycastarg(x, vararg_type.annotation), args[i:]))
        else:
            out_args.extend(args[i:])

    return out_args


def basic_command(a: int, b: float, *args):
    ic(a, b, a)


process_arg_types(basic_command, ('a', 'n', 'lol', 'testing'))

[<ERROR: invalid literal for int() with base 10: 'a'>,
 <ERROR: could not convert string to float: 'n'>,
 'lol',
 'testing']